In [654]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

import torch
from torch import nn

### read data

In [655]:
train = pd.read_csv('train2.csv')
test = pd.read_csv('test2.csv')

In [656]:
train.head()

,row_id,country,store,product,num_sold,year,quarter,month,day,dayofweek,dayofmonth,dayofyear,weekofyear,weekday,is_weekend
0,0,0,0,1,329,2015,1,1,1,3,31,1,1,3,0
1,2,0,0,2,146,2015,1,1,1,3,31,1,1,3,0
2,5,0,1,2,283,2015,1,1,1,3,31,1,1,3,0
3,8,1,0,2,250,2015,1,1,1,3,31,1,1,3,0
4,11,1,1,2,395,2015,1,1,1,3,31,1,1,3,0


### data preprocessing

In [657]:
train = train.set_index('row_id')
test = test.set_index('row_id')


In [658]:
train

,country,store,product,num_sold,year,quarter,month,day,dayofweek,dayofmonth,dayofyear,weekofyear,weekday,is_weekend
row_id,,,,,,,,,,,,,,
0,0,0,1,329,2015,1,1,1,3,31,1,1,3,0
2,0,0,2,146,2015,1,1,1,3,31,1,1,3,0
5,0,1,2,283,2015,1,1,1,3,31,1,1,3,0
8,1,0,2,250,2015,1,1,1,3,31,1,1,3,0
11,1,1,2,395,2015,1,1,1,3,31,1,1,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26282,0,0,2,238,2018,4,12,31,0,31,365,1,0,0
26285,0,1,2,360,2018,4,12,31,0,31,365,1,0,0
26288,1,0,2,351,2018,4,12,31,0,31,365,1,0,0


In [659]:
x_data = train.drop('num_sold', axis=1)
y_data = train.num_sold
std = StandardScaler()
x_data = std.fit_transform(x_data)
test = std.transform(test)

In [660]:
x_data

array([[-1.13382165, -0.81553662, -0.37187116, ..., -1.78683007,
         0.05294767, -0.59036957],
       [-1.13382165, -0.81553662,  0.98073469, ..., -1.78683007,
         0.05294767, -0.59036957],
       [-1.13382165,  1.2261865 ,  0.98073469, ..., -1.78683007,
         0.05294767, -0.59036957],
       ...,
       [ 0.03170274, -0.81553662,  0.98073469, ..., -1.78683007,
        -1.46423675, -0.59036957],
       [ 1.19722714, -0.81553662,  0.98073469, ..., -1.78683007,
        -1.46423675, -0.59036957],
       [ 1.19722714,  1.2261865 ,  0.98073469, ..., -1.78683007,
        -1.46423675, -0.59036957]])

In [661]:
x_data = torch.tensor(x_data, dtype=torch.float32)
test = torch.tensor(test, dtype=torch.float32)

y_data = torch.tensor(y_data, dtype=torch.float32)

In [662]:
print(x_data[:10])

print(x_data.shape)

tensor([[-1.1338, -0.8155, -0.3719, -1.3113, -1.4136, -1.6799, -1.6837,  0.0529,
          0.6863, -1.8128, -1.7868,  0.0529, -0.5904],
        [-1.1338, -0.8155,  0.9807, -1.3113, -1.4136, -1.6799, -1.6837,  0.0529,
          0.6863, -1.8128, -1.7868,  0.0529, -0.5904],
        [-1.1338,  1.2262,  0.9807, -1.3113, -1.4136, -1.6799, -1.6837,  0.0529,
          0.6863, -1.8128, -1.7868,  0.0529, -0.5904],
        [ 0.0317, -0.8155,  0.9807, -1.3113, -1.4136, -1.6799, -1.6837,  0.0529,
          0.6863, -1.8128, -1.7868,  0.0529, -0.5904],
        [ 0.0317,  1.2262,  0.9807, -1.3113, -1.4136, -1.6799, -1.6837,  0.0529,
          0.6863, -1.8128, -1.7868,  0.0529, -0.5904],
        [ 1.1972, -0.8155, -0.3719, -1.3113, -1.4136, -1.6799, -1.6837,  0.0529,
          0.6863, -1.8128, -1.7868,  0.0529, -0.5904],
        [ 1.1972, -0.8155,  0.9807, -1.3113, -1.4136, -1.6799, -1.6837,  0.0529,
          0.6863, -1.8128, -1.7868,  0.0529, -0.5904],
        [ 1.1972,  1.2262,  0.9807, -1.3113, -1.

### model train

In [663]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data)

In [664]:
# np.unique(y_train.detach().numpy())

In [665]:
from torch.utils.data import TensorDataset

In [666]:
from torch.utils.data import DataLoader

In [667]:
dataset_train = TensorDataset(x_train, y_train)
loader_train = DataLoader(dataset_train, batch_size=256, shuffle=True)

dataset_test = TensorDataset(x_test, y_test)
loader_test = DataLoader(dataset_test, batch_size=256, shuffle=True)


In [668]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cuda device


In [669]:
import torch.nn.functional as F
n_feature = 13
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(n_feature, n_feature*2)
        self.fc2 = nn.Linear(n_feature*2, n_feature*4)
        self.bn1 = nn.BatchNorm1d(n_feature*4)
        self.fc3 = nn.Linear(n_feature*4, n_feature*8)
        self.fc4 = nn.Linear(n_feature*8, n_feature*16)
        self.bn2 = nn.BatchNorm1d(n_feature*16)
        self.fc5 = nn.Linear(n_feature*16, n_feature*32)
        self.fc6 = nn.Linear(n_feature*32, n_feature*64)
        self.fc7 = nn.Linear(n_feature*64, n_feature*128)
        self.bn3 = nn.BatchNorm1d(n_feature*128)
        self.fc8 = nn.Linear(n_feature*128, n_feature*64)
        self.fc9 = nn.Linear(n_feature*64, n_feature*32)
        self.fc10 = nn.Linear(n_feature*32, n_feature*16)
        self.bn4 = nn.BatchNorm1d(n_feature*16)
        self.fc11 = nn.Linear(n_feature*16, n_feature*8)
        self.fc12 = nn.Linear(n_feature*8, n_feature*4)
        self.bn5 = nn.BatchNorm1d(n_feature*4)
        self.fc13 = nn.Linear(n_feature*4, n_feature*2)
        self.fc14 = nn.Linear(n_feature*2, n_feature)
        self.fc15 = nn.Linear(n_feature, 1)

        # self.fc1 = nn.Linear(n_feature, n_feature*2)
        # self.bn1 = nn.BatchNorm1d(n_feature*2)
        # self.fc2 = nn.Linear(n_feature*2, n_feature*4)
        # self.bn2 = nn.BatchNorm1d(n_feature*4)
        # self.fc3 = nn.Linear(n_feature*4, n_feature*8)
        # self.bn3 = nn.BatchNorm1d(n_feature*8)
        # self.fc4 = nn.Linear(n_feature*8, n_feature*4)
        # self.fc5 = nn.Linear(n_feature*4, n_feature*2)
        # self.bn5 = nn.BatchNorm1d(n_feature*2)
        # self.fc6 = nn.Linear(n_feature*2, n_feature)
        # self.fc7 = nn.Linear(n_feature,  1)


    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.bn1(x)

        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.bn2(x)
        x = F.relu(self.fc5(x)) 
        x = F.relu(self.fc6(x) )
        x = F.relu(self.fc7(x) )
        x = self.bn3(x) 
        x = F.relu(self.fc8(x) )
        x = F.relu(self.fc9(x) )
        x = F.relu(self.fc10(x) )
        x = self.bn4(x)
        x = F.relu(self.fc11(x) )
        x = F.relu(self.fc12(x) )
        x = self.bn5(x) 
        x = F.relu(self.fc13(x) )
        x = F.relu(self.fc14(x))
        x = self.fc15(x) 
        # x = self.fc1(x)
        # x = F.relu(self.bn1(x))
        # x = F.relu(self.fc2(x))
        # x= F.relu(self.bn2(x) )
        # x= F.relu(self.fc3(x) )
        # x= F.relu(self.bn3(x) )
        # x= F.relu(self.fc4(x) )
        # x = F.relu(self.fc5(x))
        # x = F.relu(self.bn5(x))
        # x = F.relu(self.fc6(x))
        # x = self.fc7(x)
        # x = self.fc8(x)


        
        return x

In [670]:
model = NeuralNetwork().to(device)
# model.apply(init_weights)
print(model)

NeuralNetwork(
  (fc1): Linear(in_features=13, out_features=26, bias=True)
  (fc2): Linear(in_features=26, out_features=52, bias=True)
  (bn1): BatchNorm1d(52, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc13): Linear(in_features=52, out_features=26, bias=True)
  (fc14): Linear(in_features=26, out_features=13, bias=True)
  (fc15): Linear(in_features=13, out_features=1, bias=True)
)


In [671]:
learning_rate = 0.001
batch_size = 2
epochs = 1000

In [672]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        
        X = X.to(device)
        y = y.to(device)
        
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        # scheduler.step()
        optimizer.step()

    loss = loss.item()
    loss = loss ** 0.5
    print(f"loss: {loss:>7f}")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()

    test_loss /= num_batches
    test_loss = test_loss ** 0.5
    print(f"Test Error: \n Avg loss: {test_loss:>8f} \n")

In [673]:
from math import exp
import math

In [674]:
def cyclical_lr(stepsize, min_lr=3e-4, max_lr=3e-3):
    
    # Scaler: we can adapt this if we do not want the triangular CLR
    scaler = lambda x: 1.

    # Lambda function to calculate the LR
    lr_lambda = lambda it: min_lr + (max_lr - min_lr) * relative(it, stepsize)

    # Additional function to see where on the cycle we are
    def relative(it, stepsize):
        cycle = math.floor(1 + it / (2 * stepsize))
        x = abs(it / stepsize - 2 * cycle + 1)
        return max(0, (1 - x)) * scaler(cycle)

    return lr_lambda

In [675]:
loss_fn = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
step_size = 4*len(dataset_train)
clr = cyclical_lr(step_size)
# scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, [clr])
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(loader_train, model, loss_fn, optimizer)
    test_loop(loader_test, model, loss_fn)
    # optimizer.lr = exp_decay(t)
print("Done!")

Epoch 1
-------------------------------


e:\Dev\Anaconda\lib\site-packages\torch\nn\modules\loss.py:96: UserWarning:

Using a target size (torch.Size([256])) that is different to the input size (torch.Size([256, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.

e:\Dev\Anaconda\lib\site-packages\torch\nn\modules\loss.py:96: UserWarning:

Using a target size (torch.Size([21])) that is different to the input size (torch.Size([21, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.

e:\Dev\Anaconda\lib\site-packages\torch\nn\modules\loss.py:96: UserWarning:

Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.



loss: 16.718152
Test Error: 
 Avg loss: 16.101940 

Epoch 2
-------------------------------
loss: 15.759130
Test Error: 
 Avg loss: 14.345721 

Epoch 3
-------------------------------
loss: 10.688585
Test Error: 
 Avg loss: 10.156434 

Epoch 4
-------------------------------
loss: 9.728802
Test Error: 
 Avg loss: 9.857525 

Epoch 5
-------------------------------
loss: 10.562249
Test Error: 
 Avg loss: 9.791904 

Epoch 6
-------------------------------
loss: 9.396642
Test Error: 
 Avg loss: 9.838190 

Epoch 7
-------------------------------
loss: 7.922932
Test Error: 
 Avg loss: 9.828046 

Epoch 8
-------------------------------
loss: 9.464148
Test Error: 
 Avg loss: 9.817019 

Epoch 9
-------------------------------
loss: 10.662164
Test Error: 
 Avg loss: 9.859585 

Epoch 10
-------------------------------
loss: 10.393205
Test Error: 
 Avg loss: 9.886252 

Epoch 11
-------------------------------
loss: 10.889742
Test Error: 
 Avg loss: 9.929006 

Epoch 12
-----------------------------

KeyboardInterrupt: 

In [ ]:
def SMAPE(y_true, y_pred):
    denominator = (y_true + np.abs(y_pred)) / 200.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return np.mean(diff)


In [ ]:
x_train = x_train.to(device)
x_test = x_test.to(device)

train_pred = model(x_train)
test_pred = model(x_test)

train_score = SMAPE(y_train.cpu().detach().numpy(), train_pred.cpu().detach().numpy())
test_score = SMAPE(y_test.cpu().detach().numpy(), test_pred.cpu().detach().numpy())

print('train_score ', train_score, 'test_score', test_score)

train_score  38.26346 test_score 38.459766


In [ ]:
test = test.to(device)

In [ ]:
test_pred = model(test)

In [ ]:
train_pred.cpu()

tensor([[231.3905],
        [232.1489],
        [269.8660],
        ...,
        [235.3934],
        [232.1312],
        [265.7627]], grad_fn=<ToCopyBackward0>)

In [ ]:
unique, counts = np.unique(y_train.cpu().detach().numpy(), return_counts=True)
unique_p, counts_p = np.unique(np.round(train_pred.cpu().detach().numpy()), return_counts=True)
d_p = pd.DataFrame(index = unique_p,data = counts_p,columns=['count_pred'])
d = pd.DataFrame(index= unique,data=  counts,columns=['count_true'])
d1 = d.join(d_p)
d1 = d1.fillna(0)


In [ ]:
d_p

,count_pred
207.0,1
208.0,2
209.0,9
210.0,4
211.0,12
...,...
286.0,3
287.0,8
288.0,1
289.0,1


In [ ]:
import plotly.express as px

fig = px.line(d1, x= d.index, y = ['count_true', 'count_pred'])
fig.show()

In [ ]:
submission = pd.read_csv('sample_submission.csv')
submission.num_sold = test_pred.cpu().detach().numpy().reshape(-1)
submission.to_csv('submission.csv', index=False)